# Decision Tree (의사결정 나무)

컴퓨터 알고리즘에서 즐겨 사용하는 Tree 구조를 사용하고, 각 분기점(node)에는 분석 대상의 속성들이 위치함

각 분기점마다 목표 값을 가장 잘 분류할 수 있는 속성을 찾아서 배치하고, 해당 속성이 갖는 값을 이용하여 새로운 가지를 만듦

각 분기점에서 최적의 속성을 선택할 때는 해당 속성을 기준으로 분류한 값들이 구분되는 정도를 측정함

다른 종류의 값들이 섞여 있는 정도를 나타내는 Entropy를 주로 활용하는데, Entropy가 낮을수록 분류가 잘 된것임

Entropy가 일정 수준 이하로 낮아질 때 까지 앞의 과정을 반복  



### 구현방식

재귀적 분기 -> 엔트로피 계산 -> 재귀적 분기 -> 엔트로피 계산 -> ... -> 엔트로피 계산

+ Pruning

## Import library

In [4]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics 

## Load data
유방암 진단 데이터셋

In [2]:
uci_path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/\
breast-cancer-wisconsin/breast-cancer-wisconsin.data'

In [3]:
df = pd.read_csv(uci_path, header=None)

In [5]:
df.columns = ['id','clump','cell_size','cell_shape', 'adhesion','epithlial', 'bare_nuclei','chromatin','normal_nucleoli', 'mitoses', 'class'] 

## EDA

null(nan) 데이터 처리와 object를 int형으로 변환해야 함

## Modeling

정확도가 98% 이상이 되도록 만들어주세요 !!!

종속변수는 'class'입니다.

모델은 아래를 참고하여 쓰면 됩니다.
- DecisionTreeClassifier(criterion='entropy', max_depth=3)

#### 파라미터 설명
criterion : 분류척도

max_depth : 가지치는 최대 깊이

    - max_depth가 커질수록 모델 정확도는 높아질 수 있으나, Overfitting, 즉 실제 데이터에 대해서는 정확도가 떨어질 수 있다.